# БФИ2001 Фаттахов Тагир

## Лабораторная работа №6 (Прогноз успеха фильмов по обзорам)

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.datasets import imdb
from tensorflow.keras.models import Sequential

Загрузка данных imdb 

In [2]:
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

Информация о загруженном датасете 

In [3]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Categories: [0 1]
Number of unique words: 9998
Average Review length: 234.75892
Standard Deviation: 173


Первый обзор

In [4]:
print("Label:", targets[0])
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Преобразование обзора в текст

In [5]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded)

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

Функция векторизации данных 

In [6]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

data1 = vectorize(data)
targets1 = np.array(targets).astype("float32")

Разделение данных на тестовую и обучающую выборки в отношении (1:4)

In [7]:
test_x = data1[:10000]
test_y = targets1[:10000]
train_x = data1[10000:]
train_y = targets1[10000:]

Функция создания модели ИНС

In [8]:
def create_model(dim = 10000):
    model = Sequential()
    model.add(layers.Dense(50, activation = "relu", input_shape=(dim, )))
    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(50, activation = "relu"))
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
    model.add(layers.Dense(50, activation = "relu"))
    model.add(layers.Dense(1, activation = "sigmoid"))
    return model;

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                500050    
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 505,201
Trainable params: 505,201
Non-trai

Инициализация и обучение модели с вектором равным 10000, 2 эпохами и размером batch_size = 500

In [9]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [10]:
results = model.fit(train_x, train_y, epochs= 2, batch_size = 500, validation_data = (test_x, test_y))

Epoch 1/2
80/80 [==============================] - 3s 30ms/step - loss: 0.4005 - accuracy: 0.8237 - val_loss: 0.2622 - val_accuracy: 0.8952
Epoch 2/2
80/80 [==============================] - 2s 24ms/step - loss: 0.2123 - accuracy: 0.9197 - val_loss: 0.2651 - val_accuracy: 0.8934


Инициализация и обучение модели с вектором равным 10000, 20 эпохами и размером batch_size = 500

In [11]:
model1 = create_model()
model1.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
results1 = model1.fit(train_x, train_y, epochs= 20, batch_size = 500, validation_data = (test_x, test_y))

Epoch 1/20
80/80 [==============================] - 3s 29ms/step - loss: 0.4032 - accuracy: 0.8214 - val_loss: 0.2626 - val_accuracy: 0.8943
Epoch 2/20
80/80 [==============================] - 2s 25ms/step - loss: 0.2175 - accuracy: 0.9171 - val_loss: 0.2598 - val_accuracy: 0.8942
Epoch 3/20
80/80 [==============================] - 2s 22ms/step - loss: 0.1553 - accuracy: 0.9441 - val_loss: 0.2933 - val_accuracy: 0.8888
Epoch 4/20
80/80 [==============================] - 2s 22ms/step - loss: 0.1075 - accuracy: 0.9632 - val_loss: 0.3476 - val_accuracy: 0.8838
Epoch 5/20
80/80 [==============================] - 2s 22ms/step - loss: 0.0692 - accuracy: 0.9773 - val_loss: 0.4303 - val_accuracy: 0.8848
Epoch 6/20
80/80 [==============================] - 2s 24ms/step - loss: 0.0511 - accuracy: 0.9834 - val_loss: 0.4678 - val_accuracy: 0.8802
Epoch 7/20
80/80 [==============================] - 2s 24ms/step - loss: 0.0378 - accuracy: 0.9869 - val_loss: 0.4992 - val_accuracy: 0.8783
Epoch 8/20
80

Инициализация и обучение модели с вектором равным 12000, 2 эпохами и размером batch_size = 500

In [12]:
data1 = vectorize(data, 12000)
targets1 = np.array(targets).astype("float32")
test_x1 = data1[:10000]
test_y1 = targets1[:10000]
train_x1 = data1[10000:]
train_y1 = targets1[10000:]

model2 = create_model(12000)
model2.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
results2 = model2.fit(train_x1, train_y1, epochs= 2, batch_size = 500, validation_data = (test_x1, test_y1))

Epoch 1/2
80/80 [==============================] - 6s 58ms/step - loss: 0.4063 - accuracy: 0.8197 - val_loss: 0.2649 - val_accuracy: 0.8938
Epoch 2/2
80/80 [==============================] - 2s 29ms/step - loss: 0.2115 - accuracy: 0.9193 - val_loss: 0.2674 - val_accuracy: 0.8957


Инициализация и обучение модели с вектором равным 12000, 5 эпохами и размером batch_size = 100

In [13]:
model3 = create_model(12000)
model3.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
results3 = model3.fit(train_x1, train_y1, epochs= 5, batch_size = 100, validation_data = (test_x1, test_y1))

Epoch 1/5
400/400 [==============================] - 7s 15ms/step - loss: 0.3273 - accuracy: 0.8618 - val_loss: 0.2614 - val_accuracy: 0.8938
Epoch 2/5
400/400 [==============================] - 6s 14ms/step - loss: 0.1986 - accuracy: 0.9243 - val_loss: 0.2749 - val_accuracy: 0.8910
Epoch 3/5
400/400 [==============================] - 6s 14ms/step - loss: 0.1385 - accuracy: 0.9487 - val_loss: 0.3131 - val_accuracy: 0.8882
Epoch 4/5
400/400 [==============================] - 6s 14ms/step - loss: 0.1017 - accuracy: 0.9631 - val_loss: 0.3643 - val_accuracy: 0.8844
Epoch 5/5
400/400 [==============================] - 6s 14ms/step - loss: 0.0776 - accuracy: 0.9715 - val_loss: 0.4316 - val_accuracy: 0.8782


Инициализация и обучение модели с вектором равным 20000, 2 эпохами и размером batch_size = 500

In [51]:
data2 = vectorize(data, 20000)
targets2 = np.array(targets).astype("float32")
test_x2 = data2[:10000]
test_y2 = targets2[:10000]
train_x2 = data2[10000:]
train_y2 = targets2[10000:]

model4 = create_model(20000)
model4.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
results4 = model4.fit(train_x2, train_y2, epochs= 2, batch_size = 500, validation_data = (test_x2, test_y2))

Epoch 1/2
80/80 [==============================] - 7s 74ms/step - loss: 0.4228 - accuracy: 0.8029 - val_loss: 0.2648 - val_accuracy: 0.8926
Epoch 2/2
80/80 [==============================] - 3s 42ms/step - loss: 0.2146 - accuracy: 0.9191 - val_loss: 0.2613 - val_accuracy: 0.8955


Функция по преобразованию текста в список значений

In [55]:
def text_set(text: str, dim = 10000):
    index = imdb.get_word_index()
    text = text.replace(',', '')
    text = text.replace('.', '')
    arr_text = text.lower().split()
    index_text = [index.get(word, -4) + 3 for word in arr_text]
    result = np.zeros((1,dim))
    for i in index_text:
        if(i <= dim):
            result[0,i] = 1
    return result

2 пользовательских обзора на фильм

In [45]:
negative = """
Either. something, broke or deteriorated in me
or the eternal evil cynic Tarantino got tired 
and also lost his already shaky affection for
humanity And a large amount of positive criticism
looks like inertia respect for the master and 
the search for meaning where there is none
In all absolutely all of Tarantino's films 
there was something that justified cruelty 
there were charming heroes who despite everything
were pitied they forced to empathize It was a daring story 
In the film there is none of the above 
it's three hours of tedious chatter against the background 
of very beautiful American landscapes and some kind of boundless cruelty
which for obvious reasons cannot be taken seriously 
but it's generally unclear how it relates
All the wonderful actors involved in the film seem to be puppets
and obey the will of the mad director The only casting decision
that surprised me was Channing Tatum in this rather familiar company 
His output although episodic is memorable
In general I was thinking hard after the film 
for which Tarantino films are loved To be honest 
I stopped understanding And all 
these games of symbols that cause the audience's delight 
about eights seem very superficial
"""

positive = """
As for me, this film is a film not only about
disgusting characters in the style of a
western, but also about demonstrating real directorial art.
The film is conversational, shot indoors, in 1-2 locations, 
built on dialogues and narration. And as always, 
the dialogues in this movie are just fantastic. 
Just imagine 'Pumping music is playing, crackling 
fire in the fireplace, a blizzard is blowing outside 
the window, a western, the main characters are sipping 
hot coffee and something is about to happen. And all 
this takes place in a small single wooden house 
in the middle of a whole wasteland 
This is where the action of the film unfolds. I've always
wondered how Tarantino can convey the whole 
essence of the characters to the smallest detail 
with conversations and at the same time attract 
the viewer. The music is chosen just perfectly, 
an absolutely deserved Golden Globe. And for 
so many unpredictable and fascinating scenarios, 
you can safely award an Oscar. I can't ignore 
Jennifer's simply magnificent game, which was 
simply gorgeous in this movie!
The Abominable Eight is a film in which the authors 
have turned out everything: music, script, acting, 
dialogues, camerawork, the mesmerizing winter atmosphere 
of a western. And accusing Tarantino of filming the 
same thing is like accusing a person of speaking his native language.
"""

Функция распознования настроения текста

In [58]:
def predict_class(text):
    arr = model.predict(text)
    if(arr[0,0] > 0.50):
        return "positive"
    else:
        return "negative"

b = text_set(negative)
print("film1:", predict_class(b))

b = text_set(positive)
print("film2:", predict_class(b))

1/1 [==============================] - 0s 16ms/step
film1: negative
1/1 [==============================] - 0s 16ms/step
film2: positive
